# Feature Importance
### 1) data load 및 train ~ test로 split
### 2) Model
### 3) (Feature 별로 reverse해서) 예측해서 loss값 구하기. -- 괄호는 이번 프로젝트를 위한 과정.
### 4) Display LSTM Feature Importance
### 5) Visualization (Graph)



In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
import warnings
warnings.filterwarnings(action = 'ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# 1) data load 및 split
import random

# data seed random 설정
seed_num = 42
random.seed(seed_num)

x = np.load('C:/Users/FLAG/Desktop/dsml_data/x_(7727,10,4068).npy')
y = np.load('C:/Users/FLAG/Desktop/dsml_data/y_(7727,1).npy')

# 랜덤으로 700개 추출
# random seed에 따른 shuffle

idx = list(range(len(x)))
random.shuffle(idx)
idx = idx[:700]

i = round(len(idx)*0.8)

X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((560, 10, 4068), (560,), (140, 10, 4068), (140,))

In [3]:
type(X_train)

numpy.ndarray

In [11]:
# 2) Model
from tensorflow.keras.models import Sequential # https://juun42.tistory.com/52, Error message: module 'keras.utils.generic_utils' has no attribute 'populate_dict_with_module_objects'
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping

# ----------------------
seed_num = 42
# ----------------------
tf.random.set_seed(seed_num)

lstm2 = Sequential()
lstm2.add(InputLayer(input_shape = (X_train.shape[1], X_train.shape[2])))
lstm2.add(LSTM(units=128, activation = 'hard_sigmoid', return_sequences = True))
lstm2.add(LSTM(units=64, activation = 'hard_sigmoid', return_sequences = True))
lstm2.add(Dropout(0.2))

lstm2.add(LSTM(units=64, activation = 'hard_sigmoid', return_sequences = True))
lstm2.add(LSTM(units=32, activation = 'hard_sigmoid', return_sequences = False))
lstm2.add(Dropout(0.2))

lstm2.add(Dense(units=1, activation = 'sigmoid'))

early_stop = EarlyStopping(monitor='val_acc', patience=30, verbose=1, restore_best_weights = True)
# patience : 최고 성능 나온 이후에 학습 얼마나 더 해볼지 (뒤에 또 나올 수 있으니까.)
# verbose : 0,1,2 중 선택, 훈련 진행과정 얼마나 자세히 표시할지 선택
# restore_best_weights : 학습 중에 가장 좋았던 weights 저장할 지 여부
# https://tykimos.github.io/2017/07/09/Early_Stopping/


lstm2.compile(optimizer = 'adam', loss = tf.keras.losses.BinaryCrossentropy(), metrics = ['acc'])
lstm2.fit(X_train, y_train, validation_split = 0.25, batch_size = 128, epochs = 100, callbacks=[early_stop], shuffle=False)
# validation_split : x,y train에서 일정 비율 분리해서 검증으로 사용. validation_data param 대신 사용.
# batch_size : 훈련 모델에 넣을 덩어리 크기
# epochs : 훈련 횟수
# https://roboreport.co.kr/%EB%94%A5%EB%9F%AC%EB%8B%9Dlstm%EC%9C%BC%EB%A1%9C-%EC%95%84%ED%8C%8C%ED%8A%B8-%EC%A7%80%EC%88%98-%EC%98%88%EC%B8%A1%ED%95%98%EA%B8%B0-2-lstm-%EC%8B%A4%ED%97%98%ED%95%98%EA%B8%B0/

Epoch 1/100
4/4 [==============================] - 8s 386ms/step - loss: 0.6892 - acc: 0.5786 - val_loss: 0.6639 - val_acc: 0.6214
Epoch 2/100
4/4 [==============================] - 1s 159ms/step - loss: 0.6822 - acc: 0.5714 - val_loss: 0.6675 - val_acc: 0.6214
Epoch 3/100
4/4 [==============================] - 1s 156ms/step - loss: 0.6811 - acc: 0.5714 - val_loss: 0.6684 - val_acc: 0.6214
Epoch 4/100
4/4 [==============================] - 1s 173ms/step - loss: 0.6752 - acc: 0.5881 - val_loss: 0.6668 - val_acc: 0.6214
Epoch 5/100
4/4 [==============================] - 1s 160ms/step - loss: 0.6832 - acc: 0.5476 - val_loss: 0.6653 - val_acc: 0.6214
Epoch 6/100
4/4 [==============================] - 1s 158ms/step - loss: 0.6742 - acc: 0.5833 - val_loss: 0.6647 - val_acc: 0.6214
Epoch 7/100
4/4 [==============================] - 1s 157ms/step - loss: 0.6785 - acc: 0.5905 - val_loss: 0.6647 - val_acc: 0.6214
Epoch 8/100
4/4 [==============================] - 1s 159ms/step - loss: 0.6821 - a

In [12]:
# 3) feature 별로 reverse해서 예측해서 loss값 구하기.
# cross_entropy의 방법으로는 binary_cross_entropy사용

# feaures (itemid가 피쳐임.)
from tqdm import tqdm

a = pd.read_csv('C:/Users/FLAG/Desktop/dsml_data/total_data_7727.csv')
features = list(a['ITEMID'].sort_values().unique()) #4028개의 itemid존재.

results = []
preds = lstm2.predict(X_test) # baseline위해서


# COMPUTE BASELINE
# loss값 구하기
from tensorflow.keras.losses import BinaryCrossentropy # binary cross entropy: loss 값의 종류
bce = BinaryCrossentropy()
baseline_bce = bce(y_test, preds).numpy() #true, pred를 이용해 loss값 구한 게 baseline
results.append({'feature':'BASELINE','baseline_bce':baseline_bce}) 

for k in tqdm(range(len(features))): #feature(itemid)별...for문, suffle도 되고,,, reverse도 되고
    
    # REVERSE ALL FEATURE K
    save_col = X_test[:,:,k].copy()

    ##방법1
    X_test[:,:,k] = np.abs(X_test[:,:,k]-1)

    ##방법2
    # X_test[:,:,k] = np.where(X_test[:,:,k]==1, 2, X_test[:,:,k])
    # X_test[:,:,k] = np.where(X_test[:,:,k]==0, 1, X_test[:,:,k])
    # X_test[:,:,k] = np.where(X_test[:,:,k]==2, 0, X_test[:,:,k])

    # COMPUTE BCE WITH FEATURE K REVERSED
    pred2 = lstm2.predict(X_test) #diff 위해서
    loss_bce = bce(y_test, pred2).numpy() #넘파이로 안하면 그냥 객체<tf.Tensor: shape=(), dtype=float32, numpy=1.4845252>로 나옴.

    results.append({'feature':features[k],'bce':loss_bce})
    X_test[:,:,k] = save_col #리셋

100%|██████████| 4068/4068 [10:09<00:00,  6.67it/s]


In [13]:
# 4) DISPLAY LSTM FEATURE IMPORTANCE
df = pd.DataFrame(results)
df1 = df.copy()
df1['baseline_bce'] = float(df['baseline_bce'].dropna().unique())
df1['diff'] = df1['bce']-df1['baseline_bce']
df1 = df1.sort_values(by='diff')
df1

,feature,baseline_bce,bce,diff
20,50861,1.495646,1.445308,-0.050337
3457,55390000401,1.495646,1.451920,-0.043725
3688,60505260400,1.495646,1.462788,-0.032857
3848,63323036920,1.495646,1.464371,-0.031274
3046,51079000220,1.495646,1.464618,-0.031028
...,...,...,...,...
1978,338001701,1.495646,1.544252,0.048607
2053,338519741,1.495646,1.545959,0.050313
3724,61553015311,1.495646,1.547540,0.051894
2196,409606211,1.495646,1.549407,0.053762


# D-1~ D-5 reverse

In [15]:
# 3) feature 별로 reverse해서 예측하고 loss값 구하기 
# FEATURES
a = pd.read_csv('C:/Users/FLAG/Desktop/dsml_data/total_data_7727.csv')
features = list(a['ITEMID'].sort_values().unique()) # 4068개 ITEMID 

results = []
preds = lstm2.predict(X_test)

# COMPUTE BASELINE
from tensorflow.keras.losses import BinaryCrossentropy
bce = BinaryCrossentropy()
baseline_bce = bce(y_test, preds).numpy()
results.append({'feature':'BASELINE','baseline_bce':baseline_bce}) 

for k in tqdm(range(len(features))):

    # REVERSE ALL FEATURE K
    save_col = X_test[:,:,k].copy()
    
    ##방법1
    X_test[:,5:,k] = np.abs(X_test[:,5:,k]-1)
    
    ##방법2
    # X_test[:,5:,k] = np.where(X_test[:,5:,k]==1, 2, X_test[:,5:,k])
    # X_test[:,5:,k] = np.where(X_test[:,5:,k]==0, 1, X_test[:,5:,k])
    # X_test[:,5:,k] = np.where(X_test[:,5:,k]==2, 0, X_test[:,5:,k])

    # COMPUTE BCE WITH FEATURE K REVERSED
    preds = lstm2.predict(X_test)
    loss_bce = bce(y_test, preds).numpy()

    results.append({'feature':features[k],'bce':loss_bce})
    X_test[:,:,k] = save_col

100%|██████████| 4068/4068 [10:08<00:00,  6.69it/s]


In [16]:
# 4) DISPLAY LSTM FEATURE IMPORTANCE
df2 = pd.DataFrame(results)

df_1to5 = df2.copy()
base_value = df2.iloc[0,1]
df_1to5['baseline_bce'] = base_value
df_1to5['diff'] = df_1to5['bce']-df_1to5['baseline_bce']
df_1to5 = df_1to5.drop(index=[0])
df_1to5 = df_1to5.sort_values(by='diff')
df_1to5
# df_1to5.to_csv('df_d1d5.csv',index = False)

,feature,baseline_bce,bce,diff
20,50861,1.495646,1.451938,-0.043707
3837,63323026965,1.495646,1.452003,-0.043642
3457,55390000401,1.495646,1.460225,-0.035420
1035,56017275,1.495646,1.460287,-0.035358
3968,66591018442,1.495646,1.460546,-0.035099
...,...,...,...,...
1978,338001701,1.495646,1.530784,0.035139
298,224264,1.495646,1.530957,0.035311
2053,338519741,1.495646,1.532002,0.036357
3724,61553015311,1.495646,1.534552,0.038906


# 6~10 reversed (7727)
- 1) DATA, 2) MODEL 생략

In [18]:
# 3) feature 별로 reverse해서 예측하고 loss값 구하기 
# FEATURES
a = pd.read_csv('C:/Users/FLAG/Desktop/dsml_data/total_data_7727.csv')
features = list(a['ITEMID'].sort_values().unique()) # 4068개 ITEMID 

results = []
preds = lstm2.predict(X_test)

# COMPUTE BASELINE
from tensorflow.keras.losses import BinaryCrossentropy
bce = BinaryCrossentropy()
baseline_bce = bce(y_test, preds).numpy()
results.append({'feature':'BASELINE','baseline_bce':baseline_bce}) 

for k in tqdm(range(len(features))):

    # REVERSE ALL FEATURE K
    save_col = X_test[:,:,k].copy()

    ##방법1
    X_test[:,0:5,k] = np.abs(X_test[:,0:5,k]-1)

    ##방법2
    # X_test[:,0:5,k] = np.where(X_test[:,0:5,k]==1, 2, X_test[:,0:5,k])
    # X_test[:,0:5,k] = np.where(X_test[:,0:5,k]==0, 1, X_test[:,0:5,k])
    # X_test[:,0:5,k] = np.where(X_test[:,0:5,k]==2, 0, X_test[:,0:5,k])

    # COMPUTE BCE WITH FEATURE K REVERSED
    preds = lstm2.predict(X_test)
    loss_bce = bce(y_test, preds).numpy()

    results.append({'feature':features[k],'bce':loss_bce})
    X_test[:,:,k] = save_col

100%|██████████| 4068/4068 [10:02<00:00,  6.76it/s]


In [19]:
# 4) DISPLAY LSTM FEATURE IMPORTANCE
df1 = pd.DataFrame(results)
df_6to10 = df1.copy()
df_6to10['baseline_bce'] = float(df1['baseline_bce'].dropna().unique())
df_6to10['diff'] = df_6to10['bce']-df_6to10['baseline_bce']
df_6to10 = df_6to10.sort_values(by='diff')
df_6to10 = df_6to10.drop(index=[0])
df_6to10

# df_6to10.to_csv('df_d6d10_best.csv',index = False)

,feature,baseline_bce,bce,diff
20,50861,1.495646,1.479084,-0.016561
3837,63323026965,1.495646,1.481164,-0.014481
308,224277,1.495646,1.485976,-0.009669
33,50878,1.495646,1.486513,-0.009132
3457,55390000401,1.495646,1.486525,-0.009121
...,...,...,...,...
3726,61553019648,1.495646,1.506145,0.010499
3724,61553015311,1.495646,1.507584,0.011938
574,6494300,1.495646,1.508512,0.012866
2196,409606211,1.495646,1.510602,0.014956
